In [12]:
!pip install simpletransformers --q

In [14]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 5.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.9 MB/s eta 0:00:00a 0:00:01
     ━━━

In [15]:
import data_functions

from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

import pandas as pd

/home/gundruke/micromamba/envs/aau/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# Load data from a JSON Lines (JSONL) file
filtered_train_data = data_functions.load_jsonl('data/restaurant_filtered_train.jsonl')
filtered_test_data = data_functions.load_jsonl('data/restaurant_filtered_test.jsonl')

In [17]:
aspect_map = {'ambience':0, 'food':1, 'other':2, 'price':3, 'service':4}

In [18]:
train_data = []
for d in filtered_train_data:
    asp = [0, 0, 0, 0, 0]
    for a in d['aspects']:
        asp[aspect_map[a['aspect']]] = 1
    
    train_data.append([d['text'], asp])

In [19]:
test_data = []
for d in filtered_test_data:
    asp = [0, 0, 0, 0, 0]
    for a in d['aspects']:
        asp[aspect_map[a['aspect']]] = 1
    
    test_data.append([d['text'], asp])

In [21]:
train_data[0]

['But the staff was so horrible to us.', [0, 0, 0, 0, 1]]

In [24]:
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]
train_df.head()

,text,labels
0,But the staff was so horrible to us.,"[0, 0, 0, 0, 1]"
1,"To be completely fair, the only redeeming fact...","[0, 1, 1, 0, 0]"
2,"The food is uniformly exceptional, with a very...","[0, 1, 0, 0, 0]"
3,Where Gabriela personaly greets you and recomm...,"[0, 0, 0, 0, 1]"
4,"For those that go once and don't enjoy it, all...","[0, 0, 1, 0, 0]"


In [25]:
eval_df = pd.DataFrame(test_data)
eval_df.columns = ["text", "labels"]
eval_df.head()

,text,labels
0,The bread is top notch as well.,"[0, 1, 0, 0, 0]"
1,I have to say they have one of the fastest del...,"[0, 0, 0, 0, 1]"
2,Food is always fresh and hot- ready to eat!,"[0, 1, 0, 0, 0]"
3,Did I mention that the coffee is OUTSTANDING?,"[0, 1, 0, 0, 0]"
4,"I trust the people at Go Sushi, it never disap...","[0, 0, 1, 0, 0]"


In [26]:
# Optional model configuration
model_args = MultiLabelClassificationArgs(num_train_epochs=1, overwrite_output_dir=True)

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=5,
    args=model_args,
    use_cuda=False
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df
)

config.json: 100%|██████████| 481/481 [00:00<00:00, 695kB/s]
model.safetensors: 100%|██████████| 499M/499M [01:40<00:00, 4.97MB/s] 
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'roberta.pooler.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 2.04MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 7.25MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 2.67MB/s]
  0%|          | 0/2853 [20:23<?, ?it/s]


KeyboardInterrupt: 

In [ ]:

# Make predictions with the model
predictions, raw_outputs = model.predict(["Anytime and everytime I find myself in the neighborhood I will go to Sushi Rose for fresh sushi and great portions all at a reasonable price."])